#### Load Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, glob
from datetime import datetime
import natsort
import numpy as np
from PIL import Image
from skimage import color, data, io, img_as_float
import scipy.misc as sm
import cv2

from keras import *
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras import backend as K
import h5py
from keras.optimizers import SGD, Adam, Adadelta
from keras.callbacks import *

K.set_image_dim_ordering('th')

#### Read Images and Preprocess

In [ ]:
dog_input_path = './data/dogs_input.npy'
dog_label_path = './data/dogs_label.npy'
cat_input_path = './data/cats_input.npy'
cat_label_path = './data/cats_label.npy'

dog_input = np.load(dog_input_path)
dog_label = np.load(dog_label_path)
cat_input = np.load(cat_input_path)
cat_label = np.load(cat_label_path)

dog_input = np.rollaxis(dog_input, 3, 1)
cat_input = np.rollaxis(cat_input, 3, 1)

train_dog_input = dog_input[:2500, :, :, :]
test_dog_input = dog_input[2500:, :, : :]
train_dog_label = dog_label[:2500, :]
test_dog_label = dog_label[2500:, :]

train_cat_input = cat_input[:2500, :, :, :]
test_cat_input = cat_input[2500:, :, : :]
train_cat_label = cat_label[:2500, :]
test_cat_label = cat_label[2500:, :]

train_X = np.vstack((train_dog_input, train_cat_input))
train_Y = np.vstack((train_dog_label, train_cat_label))
test_X = np.vstack((test_dog_input, test_cat_input))
test_Y = np.vstack((test_dog_label, test_cat_label))

rand_ind = np.random.permutation(np.shape(train_X)[0])
train_X = train_X[rand_ind, :, :, :]
train_Y = train_Y[rand_ind, :]

print np.shape(train_X), np.shape(train_Y), np.shape(test_X), np.shape(test_Y)

#### Model-related Functions

In [ ]:
def global_average_pooling(x):
    return K.mean(x, axis = (2, 3))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

def VGG16_convolutions():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,None,None)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    return model

def load_model_weights(model, weights_path):
    print 'Loading model.'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
        model.layers[k].trainable = False
    f.close()
    print 'Model loaded.'
    return model

def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

#### Load Model

In [ ]:
output_path = './output/'
pre_trained_model_path = './pre_trained_model/'

model = VGG16_convolutions()
model = load_model_weights(model, pre_trained_model_path + "/vgg16_weights.h5")    

model.add(Lambda(global_average_pooling, output_shape=global_average_pooling_shape))
model.add(Dense(2, activation = 'softmax', init='uniform'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=1e-6)
model.compile(loss = 'categorical_crossentropy', optimizer = adadelta, metrics=['accuracy'])

#### Train Model

In [ ]:
print "Training..."

i = datetime.now()
time_str = i.strftime('%Y-%m-%d_%H:%M:%S')

checkpoint_path="./checkpoint/dogs_and_cats_"+time_str + "_epoch_{epoch:02d}_acc_{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=0, 
                             save_best_only=True, save_weights_only=False, mode='auto')
model.fit(train_X, train_Y, nb_epoch=20, batch_size=64, verbose=1, validation_data=(test_X, test_Y), callbacks=[checkpoint])

print "Training Completed!!"

#### Visualize Class Activiation Map of Test Set

In [ ]:
#model = models.load_model('./checkpoint/dogs_and_cats_2017-02-03_12:10:18_epoch_01_acc_0.91.hdf5')

# Prepare Random Test Image
test_ind = np.random.randint(np.shape(test_X)[0])
test_input_img = test_X[test_ind:test_ind+1, :, :, :]
test_rgb_img = np.rollaxis(test_X[test_ind, :, :, :], 0, 3)
test_label = test_Y[test_ind, :]

width = np.shape(test_rgb_img)[0]
height = np.shape(test_rgb_img)[1]

#Get the 512 input weights to the softmax.
class_weights = model.layers[-1].get_weights()[0]
final_conv_layer = get_output_layer(model, "conv5_3")
get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
[conv_outputs, predictions] = get_output([test_input_img])
conv_outputs = conv_outputs[0, :, :, :]

#Create the class activation map.
cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
for i, w in enumerate(class_weights[:, 1]):
        cam += w * conv_outputs[i, :, :]
cam /= np.max(cam)
cam = cv2.resize(cam, (height, width))
heatmap = 255-cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
heatmap[np.where(cam < 0.1)] = 0

# Draw Original Image with Class Activation Map
overlap_img = Image.blend(Image.fromarray(test_rgb_img), Image.fromarray(heatmap), 0.2)
plt.imshow(np.array(overlap_img))

# Print True and Predicted Label
if test_label[0] > test_label[1]:
    print "Answer : Dog"
else: 
    print "Answer : Cat"
    
if predictions[0, 0] > predictions[0, 1]:
    print "Prediction : Dog"
else:
    print "Prediction : Cat"
    
print('Dog probability : %0.2f%%, Cat probability : %0.2f%%' % (float(100*predictions[0,0]), float(100*predictions[0,1])))